In [1]:
import pandas as pd
from utils import *
from mappings import *

In [2]:
df = read_dbf('../data/raw/members.DBF')

### Climbers

In [11]:
df.isna().sum()

expid            0
membid           0
peakid           0
myear            0
mseason          0
             ...  
mchksum          0
msmtnote1    72599
msmtnote2    72884
msmtnote3    72939
deathrte     72015
Length: 78, dtype: int64

In [8]:
df_climbers = df[['fname', 'lname', 'sex', 'yob', 'citizen', 'occupation']].drop_duplicates()
df_climbers['climber_id'] = df_climbers.groupby(['fname', 'lname', 'sex', 'yob'], as_index=False).ngroup()
df_climbers.climber_id = float_to_int(df_climbers.climber_id)

In [9]:
df_climbers.head()

,fname,lname,sex,yob,citizen,occupation,climber_id
0,Jean Robert,Clemenson,M,1938,France,Alpine guide,15951
1,Bernard,Dufour,M,1936,France,Alpine guide,3635
2,Philippe,Gerard,M,1950,France,Professional photographer,29410
3,Eric,Lasserre,M,1937,France,Physician,9319
4,Guy,Peters,M,1944,France,Alpine guide,12367


In [10]:
assert df_climbers.climber_id.isna().sum() == 0
assert df_climbers.climber_id.nunique() == df_climbers.shape[0]

AssertionError: 